# 05. Interface exercise

예전에 다뤘던 `Obj2D` 클래스는 `Shape` 클래스의 인스턴스 변수를 포함한다.
이렇게 상속 관계(is-a 관계)가 아닌 포함 관계(has-a 관계)지만 공통적으로 `display`라는 메소드로 화면에 그리는 동작을 한다.
이럴 때 활용할 수 있는 것이 바로 지난 시간에 배웠던 인터페이스다.

In [1]:
interface Displayable {
    public void display();
}

In [2]:
import io.github.spencerpark.ijava.runtime.*;
import org.apache.commons.lang3.tuple.*;

abstract class Shape implements Displayable {
    // 인스턴스 변수
    int width;  // 양과 음의 정수값 모두 가능
    int height; // 양과 음의 정수값 모두 가능
    String fill; // 도형의 안쪽을 채우는 색깔
    double opacity; // 도형을 그렸을 때 투명도
    
    Shape(int width, int height, String fill, double opacity) {
        this.width = width;
        this.height = height;
        this.fill = fill;
        this.opacity = opacity;
    }

    // 추상 메소드 
    abstract double area(); // 넓이 계산
    abstract String toSVGshape(Pair<Integer,Integer> point); // SVG 기본 도형 태그 생성
    
    @Override
    public String toString() {
        return super.toString()
            + String.format("(width=%d, height=%d, fill=%s, opacity=%f)",
                            width, height, fill, opacity );
    }
    
    @Override
    public void display() { // 이미지 형태로 보여주기 위한 메소드
        Pair<Integer,Integer> point = Pair.of( (width<0)? Math.abs(width) :0,
                                              (height<0)? Math.abs(height):0 );
        String svgStr = String.format(
            "<svg width='%d' height='%d'>%s</svg>",
            Math.abs(width), Math.abs(height), this.toSVGshape(point) );
        Display.display(svgStr,"text/html");
    }
}

In [3]:
class RightTri extends Shape {
    RightTri(int width, int height, String fill, double opacity) {
        super(width, height, fill, opacity);
    }
    
    @Override
    double area() { return Math.abs(width * height) / 2; } // 삼각형 넓이공식에 맞게
    
    @Override
    String toSVGshape(Pair<Integer,Integer> point) {
        int x0 = point.getLeft();
        int y0 = point.getRight();
        return
            String.format("<circle cx='%d' cy='%d' r='3' fill='%s' opacity='%f' />",
                          x0,y0, fill, opacity)
            +
            String.format("<polygon points='%d,%d %d,%d %d,%d' fill='%s' opacity='%f' />",
                          x0,y0, x0+width,y0, x0,y0+height, fill, opacity );
    }
}

In [4]:
class Rectangle extends Shape {
    Rectangle(int width, int height, String fill, double opacity) {
        super(width, height, fill, opacity);
    }
    
    @Override
    double area() { return Math.abs(width * height); } // 직사각형 넓이공식에 맞게
    
    @Override
    String toSVGshape(Pair<Integer,Integer> point) {
        int x0 = point.getLeft();
        int y0 = point.getRight();
        return
            String.format("<circle cx='%d' cy='%d' r='3' fill='%s' opacity='%f' />",
                          x0,y0, fill, opacity)
            +
            String.format("<rect width='%d' height='%d' fill='%s' opacity='%f' />",
                          Math.abs(width), Math.abs(height), fill, opacity );
    }
}

In [5]:
new RightTri(20,40,"red",0.3).display()

<svg width='20' height='40'><circle cx='0' cy='0' r='3' fill='red' opacity='0.300000' /><polygon points='0,0 20,0 0,40' fill='red' opacity='0.300000' /></svg>

In [6]:
new RightTri(-20,40,"green",0.3).display()

<svg width='20' height='40'><circle cx='20' cy='0' r='3' fill='green' opacity='0.300000' /><polygon points='20,0 0,0 20,40' fill='green' opacity='0.300000' /></svg>

In [7]:
new RightTri(20,-40,"purple",0.3).display()

<svg width='20' height='40'><circle cx='0' cy='40' r='3' fill='purple' opacity='0.300000' /><polygon points='0,40 20,40 0,0' fill='purple' opacity='0.300000' /></svg>

In [8]:
new RightTri(-20,-40,"brown",0.3).display()

<svg width='20' height='40'><circle cx='20' cy='40' r='3' fill='brown' opacity='0.300000' /><polygon points='20,40 0,40 20,0' fill='brown' opacity='0.300000' /></svg>

In [9]:
new Rectangle(20,40,"red",0.3).display()

<svg width='20' height='40'><circle cx='0' cy='0' r='3' fill='red' opacity='0.300000' /><rect width='20' height='40' fill='red' opacity='0.300000' /></svg>

In [10]:
new Rectangle(-20,40,"green",0.3).display()

<svg width='20' height='40'><circle cx='20' cy='0' r='3' fill='green' opacity='0.300000' /><rect width='20' height='40' fill='green' opacity='0.300000' /></svg>

In [11]:
new Rectangle(20,-40,"purple",0.3).display()

<svg width='20' height='40'><circle cx='0' cy='40' r='3' fill='purple' opacity='0.300000' /><rect width='20' height='40' fill='purple' opacity='0.300000' /></svg>

In [12]:
new Rectangle(-20,-40,"brown",0.3).display()

<svg width='20' height='40'><circle cx='20' cy='40' r='3' fill='brown' opacity='0.300000' /><rect width='20' height='40' fill='brown' opacity='0.300000' /></svg>

In [13]:
class Obj2D implements Displayable {
    Pair<Integer, Integer> point;
    Shape shape;
    
    Obj2D(Pair<Integer, Integer> point, Shape shape) {
        this.point = point;
        this.shape = shape;
    }
    
    @Override
    public String toString() {
        return
            super.toString()
            +
            String.format("( point=%s, shape=%s )",
                          point.toString(), shape.toString() );
    }
    
    String toSVGshape() { return shape.toSVGshape(point); }

    @Override
    public void display() { // 이미지 형태로 보여주기 위한 메소드
        String svgStr = String.format(
            "<svg width='%d' height='%d'>%s</svg>",
            point.getLeft()+Math.abs(shape.width),
            point.getRight()+Math.abs(shape.height),
            this.toSVGshape() );
        Display.display(svgStr,"text/html");
    }
}

In [14]:
Pair.of(150,100);

(150,100)

In [15]:
Pair.of(150,100).getClass()

class org.apache.commons.lang3.tuple.ImmutablePair

In [16]:
Obj2D otri1 = new Obj2D( Pair.of(150,100), new RightTri(30,40,"red",0.3) );
Obj2D otri2 = new Obj2D( Pair.of(150,100), new RightTri(-30,-40,"red",0.3) );

In [17]:
otri1

REPL.$JShell$26$Obj2D@4d2fd911( point=(150,100), shape=REPL.$JShell$16$RightTri@343ff194(width=30, height=40, fill=red, opacity=0.300000) )

In [18]:
otri2

REPL.$JShell$26$Obj2D@3b05af8d( point=(150,100), shape=REPL.$JShell$16$RightTri@1f96fdc8(width=-30, height=-40, fill=red, opacity=0.300000) )

In [19]:
Obj2D orect1 = new Obj2D( Pair.of(150,100), new Rectangle(30,40,"blue",0.3) );

orect1

REPL.$JShell$26$Obj2D@671b13c7( point=(150,100), shape=REPL.$JShell$17$Rectangle@67f9c52f(width=30, height=40, fill=blue, opacity=0.300000) )

In [20]:
orect1.toSVGshape()

<circle cx='150' cy='100' r='3' fill='blue' opacity='0.300000' /><rect width='30' height='40' fill='blue' opacity='0.300000' />

In [21]:
otri1.toSVGshape()

<circle cx='150' cy='100' r='3' fill='red' opacity='0.300000' /><polygon points='150,100 180,100 150,140' fill='red' opacity='0.300000' />

In [22]:
otri1.display()

<svg width='180' height='140'><circle cx='150' cy='100' r='3' fill='red' opacity='0.300000' /><polygon points='150,100 180,100 150,140' fill='red' opacity='0.300000' /></svg>

In [23]:
otri2.toSVGshape()

<circle cx='150' cy='100' r='3' fill='red' opacity='0.300000' /><polygon points='150,100 120,100 150,60' fill='red' opacity='0.300000' />

In [24]:
otri2.display()

<svg width='180' height='140'><circle cx='150' cy='100' r='3' fill='red' opacity='0.300000' /><polygon points='150,100 120,100 150,60' fill='red' opacity='0.300000' /></svg>

In [25]:
Obj2D otri3 = new Obj2D( Pair.of(20,30), new RightTri(-30,-40,"red",0.3) );

In [26]:
otri3.toSVGshape()

<circle cx='20' cy='30' r='3' fill='red' opacity='0.300000' /><polygon points='20,30 -10,30 20,-10' fill='red' opacity='0.300000' />

In [27]:
otri3.display()

<svg width='50' height='70'><circle cx='20' cy='30' r='3' fill='red' opacity='0.300000' /><polygon points='20,30 -10,30 20,-10' fill='red' opacity='0.300000' /></svg>

하나의 `Shape` 오브젝트로부터 여러 개의 `Obj2D` 오브젝트를 만들 수 있다.

In [28]:
Shape s = new RightTri(30,40,"blue",0.3);

Obj2D o1 = new Obj2D( Pair.of(10,10), s );
Obj2D o2 = new Obj2D( Pair.of(60,20), s );
Obj2D o3 = new Obj2D( Pair.of(20,60), s );

In [29]:
o1.display()

<svg width='40' height='50'><circle cx='10' cy='10' r='3' fill='blue' opacity='0.300000' /><polygon points='10,10 40,10 10,50' fill='blue' opacity='0.300000' /></svg>

In [30]:
o2.display()

<svg width='90' height='60'><circle cx='60' cy='20' r='3' fill='blue' opacity='0.300000' /><polygon points='60,20 90,20 60,60' fill='blue' opacity='0.300000' /></svg>

In [31]:
o3.display()

<svg width='50' height='100'><circle cx='20' cy='60' r='3' fill='blue' opacity='0.300000' /><polygon points='20,60 50,60 20,100' fill='blue' opacity='0.300000' /></svg>

---
이제 `Shape` 오브젝트와 `Obj2D` 오브젝트를 `Displayable`이라는 공통된 인터페이스를 통해 일괄 처리가 가능하다.

In [32]:
s instanceof Shape

true

In [33]:
s instanceof Displayable

true

In [34]:
o2 instanceof Obj2D

true

In [35]:
o2 instanceof Displayable

true

In [36]:
otri1 instanceof Obj2D

true

In [37]:
otri1 instanceof Displayable

true

In [38]:
Displayable[] ds = { s, otri1, o2 };

In [39]:
ds[0].display()

<svg width='30' height='40'><circle cx='0' cy='0' r='3' fill='blue' opacity='0.300000' /><polygon points='0,0 30,0 0,40' fill='blue' opacity='0.300000' /></svg>

In [40]:
ds[1].display()

<svg width='180' height='140'><circle cx='150' cy='100' r='3' fill='red' opacity='0.300000' /><polygon points='150,100 180,100 150,140' fill='red' opacity='0.300000' /></svg>

In [41]:
ds[2].display()

<svg width='90' height='60'><circle cx='60' cy='20' r='3' fill='blue' opacity='0.300000' /><polygon points='60,20 90,20 60,60' fill='blue' opacity='0.300000' /></svg>

In [42]:
for (int i = 0; i < ds.length; ++i)
    ds[i].display()

<svg width='30' height='40'><circle cx='0' cy='0' r='3' fill='blue' opacity='0.300000' /><polygon points='0,0 30,0 0,40' fill='blue' opacity='0.300000' /></svg>

<svg width='180' height='140'><circle cx='150' cy='100' r='3' fill='red' opacity='0.300000' /><polygon points='150,100 180,100 150,140' fill='red' opacity='0.300000' /></svg>

<svg width='90' height='60'><circle cx='60' cy='20' r='3' fill='blue' opacity='0.300000' /><polygon points='60,20 90,20 60,60' fill='blue' opacity='0.300000' /></svg>

In [43]:
for (var d : ds)
    d.display()

<svg width='30' height='40'><circle cx='0' cy='0' r='3' fill='blue' opacity='0.300000' /><polygon points='0,0 30,0 0,40' fill='blue' opacity='0.300000' /></svg>

<svg width='180' height='140'><circle cx='150' cy='100' r='3' fill='red' opacity='0.300000' /><polygon points='150,100 180,100 150,140' fill='red' opacity='0.300000' /></svg>

<svg width='90' height='60'><circle cx='60' cy='20' r='3' fill='blue' opacity='0.300000' /><polygon points='60,20 90,20 60,60' fill='blue' opacity='0.300000' /></svg>